In [ ]:
import json
from pathlib import Path
from typing import Dict, List, Tuple


REQUIRED_KEYS: dict[str, type] = {
    "global_index": int,
    "name": str,
    "markdown": str,
    "status": str,
}
EXPECTED_STATUS = "ok"        

# ----- 2. Hàm đọc tệp -----
def read_json_objects(path: Path) -> List[Tuple[int, Dict]]:
    """
    Trả về list [(line_no, object)].
    Thử lần lượt:
        1) Parse cả tệp như JSON (object hoặc list)
        2) Nếu lỗi ⇒ coi mỗi dòng là 1 JSON (JSONL / NDJSON)
    """
    text = path.read_text(encoding="utf-8").strip()
    # TH2a – file là 1 object duy nhất
    try:
        obj = json.loads(text)
        return [(1, obj)] if isinstance(obj, dict) else list(enumerate(obj, 1))
    except json.JSONDecodeError:
        pass

    # TH2b – file dạng JSON Lines
    records = []
    for ln, line in enumerate(text.splitlines(), 1):
        line = line.strip()
        if not line:
            continue
        try:
            records.append((ln, json.loads(line)))
        except json.JSONDecodeError as e:
            print(f"❌  Dòng {ln}: lỗi cú pháp JSON: {e}")
    return records

# ----- 3. Hàm kiểm tra 1 record -----
def validate_record(rec: Dict) -> List[str]:
    errs = []
    # 3a) Thiếu khóa hoặc sai kiểu
    for key, typ in REQUIRED_KEYS.items():
        if key not in rec:
            errs.append(f"thiếu trường “{key}”")
        elif not isinstance(rec[key], typ):
            errs.append(f"“{key}” phải là {typ.__name__}, hiện là {type(rec[key]).__name__}")
    # 3b) Giá trị không hợp lệ
    if rec.get("status") != EXPECTED_STATUS:
        errs.append(f"status = {rec.get('status')!r}, cần {EXPECTED_STATUS!r}")
    if isinstance(rec.get("global_index"), int) and rec["global_index"] < 0:
        errs.append("global_index phải ≥ 0")
    return errs

# ----- 4. Trình điều khiển -----
def check_file(file_path: str) -> None:
    path = Path(file_path)
    duplicates, seen = [], set()
    records = read_json_objects(path)

    for ln, rec in records:
        rec_errs = validate_record(rec)
        gi = rec.get("global_index")
        if gi in seen:
            rec_errs.append(f"trùng global_index {gi}")
            duplicates.append(gi)
        seen.add(gi)
        if rec_errs:
            print(f"\n⚠️  Lỗi trong bản ghi ở dòng {ln} (global_index={gi}):")
            for e in rec_errs:
                print("   •", e)

    print("\n— Kiểm tra xong —")
    print(f"Tổng bản ghi hợp lệ: {len(seen) - len(duplicates)}/{len(records)}")
    if duplicates:
        print("Duplicate global_index:", duplicates)

# ----- 5. Chạy thử -----
if __name__ == "__main__":
    check_file("diseases_test_final.jsonl")      # ← đổi đường dẫn tệp bạn cần kiểm tra



⚠️  Lỗi trong bản ghi ở dòng 6734 (global_index=6733):
   • “markdown” phải là str, hiện là NoneType

— Kiểm tra xong —
Tổng bản ghi hợp lệ: 10143/10143


In [2]:
import json
import shutil
from pathlib import Path

# --------- Cấu hình ---------
file_path   = Path("diseases_test_final.jsonl")      # đường dẫn tệp gốc
backup_path = file_path.with_suffix(".bak")  # tên file sao lưu

# --------- 1. Sao lưu ---------
shutil.copyfile(file_path, backup_path)
print(f"Đã sao lưu: {backup_path}")

# --------- 2. Đọc + sửa ---------
fixed_lines = []
with file_path.open("r", encoding="utf-8") as fin:
    for line_no, line in enumerate(fin, 1):
        if not line.strip():                # bỏ qua dòng trống
            fixed_lines.append(line)
            continue

        obj = json.loads(line)

        # 3. Nếu 'markdown' không phải str  →  thay thế
        if not isinstance(obj.get("markdown"), str):
            print(f"Sửa dòng {line_no} (global_index={obj.get('global_index')}) "
                  f"→ đặt markdown = \"\"")
            obj["markdown"] = ""            # hoặc 'Information not available'

        fixed_lines.append(json.dumps(obj, ensure_ascii=False) + "\n")

# --------- 4. Ghi đè tệp gốc ---------
with file_path.open("w", encoding="utf-8") as fout:
    fout.writelines(fixed_lines)

print("✅  Hoàn tất sửa lỗi và ghi lại file gốc.")


Đã sao lưu: diseases_test_final.bak
Sửa dòng 6734 (global_index=6733) → đặt markdown = ""
✅  Hoàn tất sửa lỗi và ghi lại file gốc.
